# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [2]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [3]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"




@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [5]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [6]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [7]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

17953614879150298909795

## Use this to import default tbears wallet and run the transactions


In [ ]:
password = "test1_Account"
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
print(deployer_wallet.get_address())
icon_service.get_balance(deployer_wallet.get_address())

In [ ]:
contracts_file_custom = "shubh_yeouido_contracts_20210112084238.pkl"

In [ ]:
with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

# Deploying all contracts


In [10]:

contracts = {}
deploy = ['addressProvider', 'daoFund','delegation','feeProvider','governance', 'lendingPool', 'lendingPoolCore','lendingPoolDataProvider','liquidationManager','ommToken','oToken', 'priceOracle','rewards', 'sample_token','snapshot','worker_token']
# deploy = ['sicx']
# for directory in {"score"}:
# deploy= ['rewards', 'sample_token','snapshot','worker_token']
directory=os.getcwd()
for item in deploy:
    params = {}
    if item == "sample_token":
       params = {'_name': "BridgeDollars",'_symbol':'USDb' ,'_decimals': 18}
    elif item == "omm_token":
        params = {'_initialSupply':0, '_decimals': 18}
    elif item == "worker_token":
        params = {'_initialSupply':100, '_decimals': 18}
    elif item == "sicx":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif item == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(item))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(res)
    contracts[item] = res.get('scoreAddress', '')

    

eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [11]:
pprint(contracts)

{'addressProvider': 'cxef67d9fed0a19be8a5b42c5c19ecb7027e30552f',
 'daoFund': 'cxf88d1358551774b37fb37e5f1ebd20645060b01c',
 'delegation': 'cx0ac8854f9fbe0126e830386046ed26bbb6769bf1',
 'feeProvider': 'cx97397bca3097b3d79884fb77158d7abc4cb2f960',
 'governance': 'cxcebda5e0deb98c46e6562e487648acffc446eea1',
 'lendingPool': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39',
 'lendingPoolCore': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0',
 'lendingPoolDataProvider': 'cx5b611522f541f77dfeab6a8f874e0e87db320a78',
 'liquidationManager': 'cxef8e0e57e50c5986e9affc17fb7c490b7bd06924',
 'oToken': 'cx502b1186a510a839682b449066adb6a1001c1f86',
 'ommToken': 'cx5dc583ebe474fd99db3962e32deff60c76328b61',
 'priceOracle': 'cx4e592739a3cd1bbd9f826b41518038fa2e6a35cb',
 'rewards': 'cx1c1d88f6039ebb04572d194d52e9b190f37514d6',
 'sample_token': 'cx1944afdf75860b703b48c0960b19b212a8c3ae41',
 'snapshot': 'cx835f7c1c4d97a6f2f2904da573cc8c3dd8ae9f9e',
 'worker_token': 'cx7641d36dc582ae29f1f9d9c97b825a1c75a50f20'}


## deploying otoken for usdb

In [12]:
#deploy oUSDb
params = {"_name":"OmmUSDbInterestToken","_symbol":"oUSDb"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oUSDb'] = res.get('scoreAddress', '')


contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0xb906e8754e315d8fbdc76f9853b6ea94e9cb12a2244e2e1da015328d51dc1cc9', 'blockHeight': 13622341, 'blockHash': '0x9b20336cb73e03c0123e6fc60d565e384fb8280731f15f47fcb7290305a80905', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx47549e03024584dca6eeb26b7de8f189bf2b6ff9', 'stepUsed': 1146050240, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1146050240, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

{'addressProvider': 'cxef67d9fed0a19be8a5b42c5c19ecb7027e30552f',
 'daoFund': 'cxf88d1358551774b37fb37e5f1ebd20645060b01c',
 'delegation': 'cx0ac8854f9fbe0126e830386046ed26bbb6769bf1',
 'feeProvider': 'cx97397bca3097b3d79884fb77158d7abc4cb2f960',
 'governance': 'cxcebda5e0deb98c46e6562e487648acffc446eea1',
 'lendingPool': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39',
 'lendingPoolCore': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0',
 'lendingPoolDataProvider': 'cx5b611522f541f77dfeab6a8f874e0e87db320a78',
 'liquidationManager': 'cxef8e0e57e50c5986e9affc17fb7c490b7bd06924',
 'ommToken': 'cx5dc583ebe474fd99db3962e32deff60c76328b61',
 'oToken': 'cx502b1186a510a839682b449066adb6a1001c1f86',
 'priceOracle': 'cx4e592739a3cd1bbd9f826b41518038fa2e6a35cb',
 'rewards': 'cx1c1d88f6039ebb04572d194d52e9b190f37514d6',
 'sample_token': 'cx1944afdf75860b703b48c0960b19b212a8c3ae41',
 'snapshot': 'cx835f7c1c4d97a6f2f2904da573cc8c3dd8ae9f9e',
 'worker_token': 'cx7641d36dc582ae29f1f9d9c97b825a1c75a50f20',


## Deploying otoken for icx

In [13]:
#deploy oICX 
params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX'] = res.get('scoreAddress', '')

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xfb93812c8e264299396433fd048e50354df97d77f4ad7aad6ff90edcf196ec02', 'blockHeight': 13622347, 'blockHash': '0x5a60aa388891c57a43780fbdd503807936bb71d3e0cca698b1f8eb64642afb59', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx04e21bb3d2e09b0e753af2dff0eeddb011a4be75', 'stepUsed': 1146047640, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1146047640, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'addressProvider': 'cxef67d9fed0a19be8a5b42c5c19ecb7027e30552f',
 'daoFund': 'cxf88d1358551774b37fb37e5f1ebd20645060b01c',
 'delegation': 'cx0ac8854f9fbe0126e830386046ed26bbb6769bf1',
 'feeProvider': 'cx97397bca3097b3d79884fb77158d7abc4cb2f960',
 'governance': 'cxcebda5e0deb98c46e6562e487648acffc446eea1',
 'lendingPool': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39',
 'lendingPoolCore': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0',
 'lendingPoolDataProvider': 'cx5b611522f541f77dfeab6a8f874e0e87db320a78',
 'liquidationManager': 'cxef8e0e57e50c5986e9affc17fb7c490b7bd06924',
 'ommToken': 'cx5dc583ebe474fd99db3962e32deff60c76328b61',
 'oToken': 'cx502b1186a510a839682b449066adb6a1001c1f86',
 'priceOracle': 'cx4e592739a3cd1bbd9f826b41518038fa2e6a35cb',
 'rewards': 'cx1c1d88f6039ebb04572d194d52e9b190f37514d6',
 'sample_token': 'cx1944afdf75860b703b48c0960b19b212a8c3ae41',
 'snapshot': 'cx835f7c1c4d97a6f2f2904da573cc8c3dd8ae9f9e',
 'worker_token': 'cx7641d36dc582ae29f1f9d9c97b825a1c75a50f20',


## Deploying otoken for iusdc


In [14]:
#deploy oIUSDC
params = {"_name":"IconUSDInterest","_symbol":"oIUSDC","_decimals":6}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oIUSDC'] = res.get('scoreAddress', '')

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xa233bd99563d0f02c8fe180423358ab0cb4f5e4e7a2732df0a0286f21f4a2f39', 'blockHeight': 13622355, 'blockHash': '0x0b3c00a11995f9a7433f3af776f8cec49b3524418fb237b083414e1cd665116c', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx42a4734508068d0f8edd7ef0778b515788b3f3b8', 'stepUsed': 1146051760, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1146051760, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'addressProvider': 'cxef67d9fed0a19be8a5b42c5c19ecb7027e30552f',
 'daoFund': 'cxf88d1358551774b37fb37e5f1ebd20645060b01c',
 'delegation': 'cx0ac8854f9fbe0126e830386046ed26bbb6769bf1',
 'feeProvider': 'cx97397bca3097b3d79884fb77158d7abc4cb2f960',
 'governance': 'cxcebda5e0deb98c46e6562e487648acffc446eea1',
 'lendingPool': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39',
 'lendingPoolCore': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0',
 'lendingPoolDataProvider': 'cx5b611522f541f77dfeab6a8f874e0e87db320a78',
 'liquidationManager': 'cxef8e0e57e50c5986e9affc17fb7c490b7bd06924',
 'ommToken': 'cx5dc583ebe474fd99db3962e32deff60c76328b61',
 'oToken': 'cx502b1186a510a839682b449066adb6a1001c1f86',
 'priceOracle': 'cx4e592739a3cd1bbd9f826b41518038fa2e6a35cb',
 'rewards': 'cx1c1d88f6039ebb04572d194d52e9b190f37514d6',
 'sample_token': 'cx1944afdf75860b703b48c0960b19b212a8c3ae41',
 'snapshot': 'cx835f7c1c4d97a6f2f2904da573cc8c3dd8ae9f9e',
 'worker_token': 'cx7641d36dc582ae29f1f9d9c97b825a1c75a50f20',


## Other Contract Addresses

In [15]:
contracts['sicx']="cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281"
contracts['staking']="cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c"
contracts['iusdc']="cx65f639254090820361da483df233f6d0e69af9b7"
contracts['usdb']="cxaa068556df80f9917ef146e889f0b2c4b13ab634"


In [16]:
pprint(contracts)

{'addressProvider': 'cxef67d9fed0a19be8a5b42c5c19ecb7027e30552f',
 'daoFund': 'cxf88d1358551774b37fb37e5f1ebd20645060b01c',
 'delegation': 'cx0ac8854f9fbe0126e830386046ed26bbb6769bf1',
 'feeProvider': 'cx97397bca3097b3d79884fb77158d7abc4cb2f960',
 'governance': 'cxcebda5e0deb98c46e6562e487648acffc446eea1',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39',
 'lendingPoolCore': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0',
 'lendingPoolDataProvider': 'cx5b611522f541f77dfeab6a8f874e0e87db320a78',
 'liquidationManager': 'cxef8e0e57e50c5986e9affc17fb7c490b7bd06924',
 'oICX': 'cx04e21bb3d2e09b0e753af2dff0eeddb011a4be75',
 'oIUSDC': 'cx42a4734508068d0f8edd7ef0778b515788b3f3b8',
 'oToken': 'cx502b1186a510a839682b449066adb6a1001c1f86',
 'oUSDb': 'cx47549e03024584dca6eeb26b7de8f189bf2b6ff9',
 'ommToken': 'cx5dc583ebe474fd99db3962e32deff60c76328b61',
 'priceOracle': 'cx4e592739a3cd1bbd9f826b41518038fa2e6a35cb',
 'rewards': 'cx1c1d

## Saving the pkl file


In [18]:
now = datetime.utcnow()
filename =network +f' contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
# filename = "kailash_custom.pkl"
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:

    pkl.dump(contracts, f)

saving contracts list as yeouido contracts_20210301113214.pkl


## Updating contracts

In [54]:
contract_ = 'snapshot'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

update snapshot cx835f7c1c4d97a6f2f2904da573cc8c3dd8ae9f9e
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xf57f426bb4a83e2ea3816f54f888e31beab53c798e06e0b0c7f2c66c98fd0821',
 'blockHeight': 13624110,
 'blockHash': '0x0f6455f31582c75e9713d3973f3442d4137e14908d9dcf34070c441266de9d00',
 'txIndex': 1,
 'to': 'cx835f7c1c4d97a6f2f2904da573cc8c3dd8ae9f9e',
 'scoreAddress': 'cx835f7c1c4d97a6f2f2904da573cc8c3dd8ae9f9e',
 'stepUsed': 1670029600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1670029600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Configuring SCOREs's parameters

In [19]:
settings_lendingPool = [{'contract': 'lendingPool', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
                        {'contract': 'lendingPool', 'method': 'setLendingPoolDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
                        {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}},
                        {'contract': 'lendingPool', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
                        {'contract': 'lendingPool', 'method': 'setStaking', 'params':{'_address': contracts['staking']}},
                        {'contract': 'lendingPool', 'method': 'setOICX', 'params':{'_address': contracts['oICX']}},
                        {'contract': 'lendingPool', 'method': 'setReward','params': {'_address':contracts['rewards']}},
                        {'contract': 'lendingPool', 'method': 'setSICX','params': {'_address':contracts['sicx']}},
                       ]
for sett in settings_lendingPool:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))



Calling setLendingPoolCore, with parameters {'_address': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0'} on the lendingPool contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x42d1dff250ad7c00ccb9f5b1b72be6dec83f79a6425579495c002f81b90b49e1', 'blockHeight': 13622397, 'blockHash': '0x563c3168f5c5f96b9a2742c78019c2f84198e8990223b9d32bc488a25933c4e1', 'txIndex': 1, 'to': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39', 'stepUsed': 151320, 'stepPrice': 10000000000, 'cumulativeStepUsed': 151320, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [20]:
settings_lendinPoolDataProvider=[{'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['usdb'],'_sym':"USDb"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['sicx'],'_sym':"ICX"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['iusdc'],'_sym':"USDC"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setPriceOracle', 'params':{'_address': contracts['priceOracle']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPool', 'params':{'_address': contracts['lendingPool']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationManager', 'params':{'_address': contracts['liquidationManager']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setStaking', 'params':{'_address': contracts['staking']}}
            ]
for sett in settings_lendinPoolDataProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setSymbol, with parameters {'_reserve': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634', '_sym': 'USDb'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setSymbol, with parameters {'_reserve': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281', '_sym': 'ICX'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while 

In [21]:
settings_feeprovider =[{'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}}]
for sett in settings_feeprovider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLoanOriginationFeePercentage, with parameters {'_percentage': 1000000000000000} on the feeProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


In [22]:
settings_oToken =[{'contract': 'oUSDb', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oUSDb', 'method': 'setReserve', 'params':{'_address':contracts['usdb']}},
            {'contract': 'oUSDb', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oUSDb', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}}]
for sett in settings_oToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0'} on the oUSDb contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634'} on the oUSDb contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -3

In [33]:
setting_priceOracle =[{'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
                      {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'ICX','_quote':'USD','_rate':10*10**17}},
                      {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'IUSDC','_quote':'USDC','_rate':10*10**17}}

                      ]
for sett in setting_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling set_reference_data, with parameters {'_base': 'USDb', '_quote': 'USD', '_rate': 1000000000000000000} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling set_reference_data, with parameters {'_base': 'ICX', '_quote': 'USD', '_rate': 1000000000000000000} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom respo

In [24]:
setting_addressProvider =[  {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
                            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['usdb']}},
                            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oUSDb']}},
                            {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
                            {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
                            {'contract': 'addressProvider', 'method': 'setStaking', 'params':{'_address':contracts['staking']}},
                            {'contract': 'addressProvider', 'method': 'setIUSDC', 'params':{'_address':contracts['iusdc']}},
                            {'contract': 'addressProvider', 'method': 'setoIUSDC', 'params':{'_address':contracts['oIUSDC']}},
                            {'contract': 'addressProvider', 'method': 'setOmmToken', 'params':{'_address':contracts['ommToken']}},
                            {'contract': 'addressProvider', 'method': 'setDelegation', 'params':{'_address':contracts['delegation']}},
                            {'contract': 'addressProvider', 'method': 'setRewards', 'params':{'_address':contracts['rewards']}}
                         ]
for sett in setting_addressProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPool, with parameters {'_address': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39'} on the addressProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setLendingPoolDataProvider, with parameters {'_address': 'cx5b611522f541f77dfeab6a8f874e0e87db320a78'} on the addressProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setUSDb, with parameters {'_address': 'cxaa

In [25]:
settings_reserves = [{'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['usdb'],
                                                   "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"6{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}},
                     {'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['sicx'],
                                                    "optimalUtilizationRate":f"8{'0'*17}",
                                                    "baseBorrowRate":f"0{'0'*17}",
                                                    "slopeRate1":f"8{'0'*16}",
                                                    "slopeRate2":f"2{'0'*18}"} ]}},
                    {'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['iusdc'],
                                                    "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"6{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}}
                    ]
for sett in settings_reserves:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setReserveConstants, with parameters {'_constants': [{'reserve': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634', 'optimalUtilizationRate': '800000000000000000', 'baseBorrowRate': '20000000000000000', 'slopeRate1': '60000000000000000', 'slopeRate2': '1000000000000000000'}]} on the lendingPoolCore contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserveConstants, with parameters {'_constants': [{'reserve': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281', 'optimalUtilizationRate': '800000000000000000', 'baseBorrowRate': '000000000000000000', 'slopeRate1': '80000000000000000

In [26]:
settings_oicx = [{'contract': 'oICX', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
                {'contract': 'oICX', 'method': 'setReserve', 'params':{'_address':contracts['sicx']}},
                {'contract': 'oICX', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                {'contract': 'oICX', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}}]
for sett in settings_oicx:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0'} on the oICX contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281'} on the oICX contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -326

In [27]:
settings_oiusdc = [ {'contract': 'oIUSDC', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
                    {'contract': 'oIUSDC', 'method': 'setReserve', 'params':{'_address':contracts['iusdc']}},
                    {'contract': 'oIUSDC', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                    {'contract': 'oIUSDC', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}}]
for sett in settings_oiusdc:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0'} on the oIUSDC contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cx65f639254090820361da483df233f6d0e69af9b7'} on the oIUSDC contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': 

In [28]:
settings_liquidationManager =  [{'contract': 'liquidationManager', 'method': 'setLendingPoolDataProvider', 'params': {'_address':contracts['lendingPoolDataProvider']}},
                                {'contract': 'liquidationManager', 'method': 'setLendingPoolCore', 'params': {'_address':contracts['lendingPoolCore']}},
                                {'contract': 'liquidationManager', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
                                {'contract': 'liquidationManager', 'method': 'setFeeProvider','params': {'_address':contracts['feeProvider']}}
                                ]
for sett in settings_liquidationManager:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolDataProvider, with parameters {'_address': 'cx5b611522f541f77dfeab6a8f874e0e87db320a78'} on the liquidationManager contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setLendingPoolCore, with parameters {'_address': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0'} on the liquidationManager contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setPriceOracle, with parameters {

In [29]:
settings_delegation =[{'contract': 'delegation', 'method': 'setLendingPoolCore','params': {'_address':contracts['lendingPoolCore']}},
                     {'contract':  'delegation', 'method': 'setOmmToken','params': {'_address':contracts['ommToken']}}]
for sett in settings_delegation:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0'} on the delegation contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setOmmToken, with parameters {'_address': 'cx5dc583ebe474fd99db3962e32deff60c76328b61'} on the delegation contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: 

In [30]:
settings_ommToken =[{'contract': 'ommToken', 'method': 'setAdmin','params': {'_admin':contracts['rewards']}},
                    {'contract':  'ommToken', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
                    {'contract':  'ommToken', 'method': 'setRewards','params': {'_address':contracts['rewards']}},
                    {'contract':  'ommToken', 'method': 'setMinimumStake','params': {'_min':100 * 10**18}},
                    {'contract':  'ommToken', 'method': 'setUnstakingPeriod','params': {'_time': 120}}]
for sett in settings_ommToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))



Calling setAdmin, with parameters {'_admin': 'cx1c1d88f6039ebb04572d194d52e9b190f37514d6'} on the ommToken contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x30db69a354a4e4edcf82668c84c947053125b21dc3c7847348d4cd915b562cb3', 'blockHeight': 13622624, 'blockHash': '0x715ba1c287815cf8a1101563223c3b0e0c7b760e41671188269cb9d88fa04d63', 'txIndex': 1, 'to': 'cx5dc583ebe474fd99db3962e32deff60c76328b61', 'stepUsed': 148920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 148920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [32]:
settings_lendingPoolCore =[ {'contract': 'lendingPoolCore', 'method': 'setLendingPool','params': {'_address':contracts['lendingPool']}},
                            {'contract': 'lendingPoolCore', 'method': 'setDaoFund','params': {'_address':contracts['daoFund']}},
                            {'contract': 'lendingPoolCore', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
                            {'contract': 'lendingPoolCore', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
                            {'contract': 'lendingPoolCore', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
                            {'contract': 'lendingPoolCore', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}},
                            {'contract': 'lendingPoolCore', 'method': 'setStaking','params': {'_address':contracts['staking']}},
                            {'contract': 'lendingPoolCore', 'method': 'set_id','params': {'_value':'1'}},
                            {'contract': 'lendingPoolCore', 'method': 'setSymbol', 'params':{'_reserve': contracts['usdb'],'_sym':"USDb"}},
                            {'contract': 'lendingPoolCore', 'method': 'setSymbol', 'params':{'_reserve': contracts['sicx'],'_sym':"ICX"}},
                            {'contract': 'lendingPoolCore', 'method': 'setSymbol', 'params':{'_reserve': contracts['iusdc'],'_sym':"USDC"}}]
for sett in settings_lendingPoolCore:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPool, with parameters {'_address': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39'} on the lendingPoolCore contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setDaoFund, with parameters {'_address': 'cxf88d1358551774b37fb37e5f1ebd20645060b01c'} on the lendingPoolCore contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error mess

In [34]:
settings_priceOracle =[{'contract':  'priceOracle', 'method': 'setBandOracle','params': {'_address':"cx61a36e5d10412e03c907a507d1e8c6c3856d9964"}},
                       {'contract':  'priceOracle', 'method': 'toggleOraclePriceBool','params':{}}]
for sett in settings_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setBandOracle, with parameters {'_address': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling toggleOraclePriceBool, with parameters {} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'

In [56]:
pprint(contracts)

{'addressProvider': 'cxef67d9fed0a19be8a5b42c5c19ecb7027e30552f',
 'daoFund': 'cxf88d1358551774b37fb37e5f1ebd20645060b01c',
 'delegation': 'cx0ac8854f9fbe0126e830386046ed26bbb6769bf1',
 'feeProvider': 'cx97397bca3097b3d79884fb77158d7abc4cb2f960',
 'governance': 'cxcebda5e0deb98c46e6562e487648acffc446eea1',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39',
 'lendingPoolCore': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0',
 'lendingPoolDataProvider': 'cx5b611522f541f77dfeab6a8f874e0e87db320a78',
 'liquidationManager': 'cxef8e0e57e50c5986e9affc17fb7c490b7bd06924',
 'oICX': 'cx04e21bb3d2e09b0e753af2dff0eeddb011a4be75',
 'oIUSDC': 'cx42a4734508068d0f8edd7ef0778b515788b3f3b8',
 'oToken': 'cx502b1186a510a839682b449066adb6a1001c1f86',
 'oUSDb': 'cx47549e03024584dca6eeb26b7de8f189bf2b6ff9',
 'ommToken': 'cx5dc583ebe474fd99db3962e32deff60c76328b61',
 'priceOracle': 'cx4e592739a3cd1bbd9f826b41518038fa2e6a35cb',
 'rewards': 'cx1c1d

In [57]:
settings_snapshot =[{'contract': 'snapshot', 'method': 'setAdmin','params': {'_address':contracts['lendingPoolCore']}},
                    {'contract': 'snapshot', 'method': 'setGovernance','params': {'_address':contracts['governance']}}]
for sett in settings_snapshot:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setAdmin, with parameters {'_address': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0'} on the snapshot contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setGovernance, with parameters {'_address': 'cxcebda5e0deb98c46e6562e487648acffc446eea1'} on the snapshot contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32

In [46]:
settings_rewards=[ {'contract': 'rewards', 'method': 'setLendingPool','params': {'_address':contracts['lendingPool']}},
            {'contract': 'rewards', 'method': 'setOmm','params': {'_address':contracts['ommToken']}},
            {'contract': 'rewards', 'method': 'setLendingPoolCore','params': {'_address':contracts['lendingPoolCore']}},
            {'contract': 'rewards', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}},
            {'contract': 'rewards', 'method': 'setWorkerToken','params': {'_address':contracts['worker_token']}},
            {'contract': 'rewards', 'method': 'setAdmin','params': {'_address':contracts['governance']}},
            {'contract': 'rewards', 'method': 'setDaoFund','params': {'_address':contracts['daoFund']}},
            {'contract': 'rewards', 'method': 'setLpToken','params': {'_address':"cx291dacbb875a94b364194a5febaac4e6318681f7"}},
                  
            ]
for sett in settings_rewards:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPool, with parameters {'_address': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39'} on the rewards contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setOmm, with parameters {'_address': 'cx5dc583ebe474fd99db3962e32deff60c76328b61'} on the rewards contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602

KeyboardInterrupt: 

In [53]:
settings_governance =[{'contract': 'governance', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}},
                      {'contract': 'governance', 'method': 'setRewards','params': {'_address':contracts['rewards']}},
                      {'contract': 'governance', 'method': 'setStartTimestamp','params': {}}]
for sett in settings_governance:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setSnapshot, with parameters {'_address': 'cx835f7c1c4d97a6f2f2904da573cc8c3dd8ae9f9e'} on the governance contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setRewards, with parameters {'_address': 'cx1c1d88f6039ebb04572d194d52e9b190f37514d6'} on the governance contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code':

## Testing deposit

In [58]:
params = {"_amount": 5 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(5 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x6d96093d3f157e8701dcb22660a2782cc8e91e3b38c9084a9fbdeb5fe4ee92b2',
 'blockHeight': 13624196,
 'blockHash': '0x82651221c4dc28cd2fc01e4ef85f43293e653d653529abbb0a273681d8331fad',
 'txIndex': 1,
 'to': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39',
 'stepUsed': 1388900,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1388900,
 'eventLogs': [{'scoreAddress': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx0287ab61ae0bec1af11ccb108257b7d292422d39',
    'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c',
    '0x4563918244f40000'],
   'data': []},
  {'scoreAddress': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
    'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0',
    '0x440a514dc44015cb'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
   'indexed': ['Mint(Address,int,bytes)',
    'cx

In [ ]:
local_settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**16}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolAddress', 'params':{'_address': contracts['lendingPool']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationAddress', 'params':{'_address': contracts['liquidationManager']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token'],"optimalUtilizationRate":f"8{'0'*17}","baseBorrowRate":f"2{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"1{'0'*18}"} ]}},
              {'contract': 'lendingPool', 'method': 'setSICXAddress', 'params':{'_address': contracts['sicx']}},
                {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'sICX','_quote':'USD','_rate':5*10**17}},
                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sicx'],'_sym':"sICX"}},
                {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
                {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
                {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sicx'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"0{'0'*17}","slopeRate1":f"8{'0'*16}","slopeRate2":f"2{'0'*18}"} ]}},
                {'contract': 'oICX', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
                {'contract': 'oICX', 'method': 'setReserveAddress', 'params':{'_address':contracts['sicx']}},
                {'contract': 'oICX', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                {'contract': 'oICX', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
                {'contract': 'lendingPool', 'method': 'setLiquidationManagerAddress',
'params': {'_address':contracts['liquidationManager']}},
                {'contract': 'liquidationManager', 'method': 'setDataProviderAddress', 'params': {'_address':contracts['lendingPoolDataProvider']}},
                    {'contract': 'liquidationManager', 'method': 'setCoreAddress', 'params': {'_address':contracts['lendingPoolCore']}},
                {'contract': 'liquidationManager', 'method': 'setOracleAddress',
'params': {'_address':contracts['priceOracle']}},
                {'contract': 'liquidationManager', 'method': 'setFeeProviderAddress',
'params': {'_address':contracts['feeProvider']}},]

In [ ]:
for sett in local_settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

## Adding USDb reserve in LendingPoolCore



In [40]:
params ={"_reserve": {"reserveAddress":contracts['usdb'],"oTokenAddress":contracts['oUSDb'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xee5584f47733020dfc44181e10d862fe569786ad17773fb63d882ebb12d06dd3',
 'blockHeight': 13622907,
 'blockHash': '0x0891e9a2da0c4ab21a47de9c11ace73caa27f1f73b89ff40fc9f4fab0a33564e',
 'txIndex': 1,
 'to': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0',
 'stepUsed': 277040,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 277040,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding ICX reserve

In [41]:
params ={"_reserve": {"reserveAddress":contracts['sicx'],"oTokenAddress":contracts['oICX'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xe01bdb411a6f92f5e9cdf17f6185242f59adf5232ee18df8cf56750b0195324f',
 'blockHeight': 13622914,
 'blockHash': '0x641c82cf03789bcd0a2dfdc5034c53b229cce8b4b7ad0706a959c2a456255462',
 'txIndex': 1,
 'to': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0',
 'stepUsed': 276800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 276800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding IUSDC reserve

In [42]:
params ={"_reserve": {"reserveAddress":contracts['iusdc'],"oTokenAddress":contracts['oIUSDC'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"6","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xf04b0b71b58f363c4a09b3ab3eec7d04e7a945576f4d3b27680c97881796acfd',
 'blockHeight': 13622920,
 'blockHash': '0x90be468a3249c369f959ab42bc748e98556f262ceff9b69c91ac0bb2b7391eb4',
 'txIndex': 1,
 'to': 'cx4d5ce0111659955ec88bdf31b550eaae1869ceb0',
 'stepUsed': 276600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 276600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Transfering USDb to wallets

In [ ]:
private2="b3d4e18d733f05dc21f326ba4fe150514874327c738c4a19ddd357d0e8e2efed"
deployer_wallet2 = KeyWallet.load(bytes.fromhex(private2))
print(deployer_wallet2.get_address())

private3="c3d5e9afe753a07c983bb14e34f73907e40a997793ce2f7d2895cddec27f81c6"
deployer_wallet3 = KeyWallet.load(bytes.fromhex(private3))
print(deployer_wallet3.get_address())

private4="2ffb9ef51826d555189d4804161058036cf87ae8290f1a308a79d045f6cdd428"
deployer_wallet4 = KeyWallet.load(bytes.fromhex(private4))
print(deployer_wallet4.get_address())
wallets=[deployer_wallet2,deployer_wallet3,deployer_wallet4]

In [ ]:
for wallet in wallets:
    params = {"_to": wallet.get_address(), "_value": 1000000 * 10 ** 18} 
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts['sample_token'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(3)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    get_tx_result(tx_hash)

In [ ]:
pprint(contracts)

In [ ]:
params = {'_owner': "hx2176d13280d465b68afbf7f9be758e966f654629"}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)


print(int(response,0))

In [ ]:
params = {"_to": "hx2176d13280d465b68afbf7f9be758e966f654629", "_value": 1000000 * 10 ** 18}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(get_tx_result(tx_hash))

In [ ]:
for wallet in wallets:
    print(wallet.get_address())
    params = {"_to": wallet.get_address(), "_value": 1000000 * 10 ** 18}
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts['sicx'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(3)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(get_tx_result(tx_hash))

## Transfering icx
 (the below cells are optional,if you want to tranfer icx to any of the wallet than only you need to run the cells)


In [ ]:
password = "test1_Account"
icx_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
icx_wallet.get_address()
icon_service.get_balance(icx_wallet.get_address())

# run the below cell only when you want to transfer ICX to a wallet
change the wallet address and the amount of icx to be transferred to that wallet

In [ ]:
# hxd29037f1eb8c3f0e8630d797edc7410507c0089a
# hx66ee5acace86d9ce23127599b7c9daae70c9d7c1
# hx65902dd4139d04765534a4539319be874ad7aa0b
wallet = "hx2176d13280d465b68afbf7f9be758e966f654629" 
amount = 100000*10**18                               #change the amount of icx to transfer here
transaction = TransactionBuilder()\
    .from_(icx_wallet.get_address())\
    .to(wallet)\
    .nid(3) \
    .nonce(100) \
    .value(amount)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(transaction, icx_wallet,step_limit)


# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

In [ ]:
contract_ = 'lendingPool'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

In [ ]:
print(contracts) cx2d844c9613b5f864f5dace7fed82997c4c62ef01

In [ ]:
print(deployer_wallet.get_address())
params = {"_amount":   10**18 }
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool'])\
    .value(10**18)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("deposit")\
    .params(params)\
    .build()

signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

In [ ]:
params = {"_amount": 10**18 ,"_reserve":contracts['sicx'] }
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("borrow")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

In [ ]:
contracts['sicx']